# Кластеризация обращений с использованием векторных представлений

Этот notebook выполняет анализ обращений с помощью:
- Векторизации текста через Yandex Embeddings API
- Кластеризации методом K-means
- Анализа результатов кластеризации


## Импорт библиотек и настройка


In [ ]:
!pip install pandas numpy scikit-learn pickle

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from collections import Counter
import warnings
import pickle
import os
import sys
warnings.filterwarnings('ignore')

# Проверяем аргументы командной строки
FORCE_RECACHE = '--recache' in sys.argv


## Настройка параметров API

**ВНИМАНИЕ:** Замените значения ниже на ваши реальные данные для работы с Yandex Cloud API


In [ ]:
# === Настройки ===
APPEALS_FILE = "Обращения.txt"
VECTORS_CACHE_FILE = "appeals_vectors_cache.pkl"

# ЗАМЕНИТЕ НА ВАШИ ДАННЫЕ:
YANDEX_FOLDER_ID = "YOUR_FOLDER_ID_HERE"
IAM_TOKEN = "YOUR_IAM_TOKEN_HERE"

EMBEDDING_URL = "https://llm.api.cloud.yandex.net/foundationModels/v1/textEmbedding"
MODEL_URI = f"emb://{YANDEX_FOLDER_ID}/text-search-query/latest"


## Функция для получения эмбеддинга


In [ ]:
def get_embedding(text: str) -> list[float]:
    """Получение вектора через Yandex Embeddings API"""
    headers = {
        "Authorization": f"Bearer {IAM_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "modelUri": MODEL_URI,
        "text": text
    }
    try:
        response = requests.post(EMBEDDING_URL, json=payload, headers=headers)
        if response.status_code != 200:
            raise Exception(f"Ошибка Yandex API: {response.status_code} - {response.text}")
        return response.json()["embedding"]
    except Exception as e:
        print(f"Ошибка при получении эмбеддинга для текста: {text[:50]}...")
        print(f"Детали ошибки: {e}")
        return None


## Загрузка обращений


In [ ]:
print("Загружаем обращения...")
with open(APPEALS_FILE, 'r', encoding='utf-8') as f:
    appeals = [line.strip() for line in f.readlines() if line.strip()]

print(f"Найдено {len(appeals)} обращений")


## Векторизация обращений

Этот этап может занять значительное время в зависимости от количества обращений. Используется кэширование для ускорения повторных запусков.


In [ ]:
# === Проверка кэша векторов ===
if os.path.exists(VECTORS_CACHE_FILE) and not FORCE_RECACHE:
    print("Найден кэш векторов, загружаем...")
    with open(VECTORS_CACHE_FILE, 'rb') as f:
        cache_data = pickle.load(f)
        vectors = cache_data['vectors']
        successful_appeals = cache_data['appeals']
    print(f"Загружено {len(vectors)} векторов из кэша")
else:
    if FORCE_RECACHE:
        print("Принудительное обновление кэша...")
    else:
        print("Кэш не найден, начинаем векторизацию обращений...")
    vectors = []
    successful_appeals = []

    for idx, appeal in enumerate(appeals, 1):
        print(f"Обработка {idx}/{len(appeals)}: {appeal[:50]}...")
        
        embedding = get_embedding(appeal)
        if embedding is not None:
            vectors.append(embedding)
            successful_appeals.append(appeal)
        else:
            print(f"Пропущено обращение {idx}")
        
        time.sleep(0.1)  # Избегаем лимитов API

    print(f"Успешно векторизовано {len(vectors)} обращений")
    
    # Сохраняем кэш
    print("Сохраняем векторы в кэш...")
    cache_data = {
        'vectors': vectors,
        'appeals': successful_appeals
    }
    with open(VECTORS_CACHE_FILE, 'wb') as f:
        pickle.dump(cache_data, f)
    print("Кэш сохранен!")

if len(vectors) == 0:
    print("Ошибка: не удалось получить ни одного вектора")
    raise Exception("Нет векторов для анализа")


## Подготовка данных для кластеризации


In [ ]:
# === Подготовка данных ===
vectors_array = np.array(vectors)
print(f"Размер матрицы векторов: {vectors_array.shape}")


## Кластеризация методом K-means


In [ ]:
# === Кластеризация ===
optimal_k = 20  # Фиксированное количество кластеров
print(f"Выполняем кластеризацию с k={optimal_k}...")
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(vectors_array)

# Вычисляем silhouette score для оценки качества кластеризации
silhouette_avg = silhouette_score(vectors_array, cluster_labels)
print(f"Silhouette score для {optimal_k} кластеров: {silhouette_avg:.3f}")


## Создание DataFrame для анализа


In [ ]:
# === Создание DataFrame для анализа ===
df_results = pd.DataFrame({
    'appeal': successful_appeals,
    'cluster': cluster_labels
})

print(f"DataFrame создан с {len(df_results)} записями")
df_results.head()


## Статистика по кластерам


In [ ]:
# === Статистика по кластерам ===
print("=== СТАТИСТИКА ПО КЛАСТЕРАМ ===")
cluster_stats = df_results.groupby('cluster').agg({
    'appeal': 'count'
}).rename(columns={'appeal': 'count'})

cluster_stats['percentage'] = (cluster_stats['count'] / len(df_results) * 100).round(2)
print(cluster_stats)


## Примеры обращений по кластерам


In [ ]:
# === Примеры обращений по кластерам ===
print("=== ПРИМЕРЫ ОБРАЩЕНИЙ ПО КЛАСТЕРАМ ===")
for cluster_id in sorted(df_results['cluster'].unique()):
    cluster_appeals = df_results[df_results['cluster'] == cluster_id]['appeal'].tolist()
    print(f"\nКластер {cluster_id} ({len(cluster_appeals)} обращений):")
    for i, appeal in enumerate(cluster_appeals[:3]):  # Показываем первые 3
        print(f"  {i+1}. {appeal[:100]}...")


## Дополнительная статистика


In [ ]:
# === Дополнительная статистика ===
print("=== ДОПОЛНИТЕЛЬНАЯ СТАТИСТИКА ===")
print(f"Общее количество обращений: {len(successful_appeals)}")
print(f"Количество кластеров: {optimal_k} (фиксированное)")
print(f"Средний размер кластера: {len(successful_appeals)/optimal_k:.1f}")
print(f"Максимальный размер кластера: {cluster_counts.max()}")
print(f"Минимальный размер кластера: {cluster_counts.min()}")
print(f"Silhouette score: {silhouette_avg:.3f}")


## Сохранение результатов


In [ ]:
# === Сохранение результатов ===
print("Сохраняем результаты...")
df_results.to_csv('appeals_clustering_results.csv', index=False, encoding='utf-8')
cluster_stats.to_csv('cluster_statistics.csv', encoding='utf-8')

print("✅ Анализ завершен!")
print("Файлы созданы:")
print("- appeals_clustering_results.csv - детальные результаты")
print("- cluster_statistics.csv - статистика по кластерам")
print("- appeals_vectors_cache.pkl - кэш векторов (для быстрого перезапуска)")
print("\nДля принудительного обновления кэша используйте: python vectorize_and_cluster_appeals.py --recache")


## Заключение

Этот notebook выполняет анализ обращений с помощью векторных представлений:

1. **Векторизация**: Преобразование текстовых обращений в числовые векторы через Yandex Embeddings API
2. **Кластеризация**: Группировка обращений в тематические кластеры с помощью K-means
3. **Анализ**: Статистический анализ результатов кластеризации

### Основные результаты:
- Кластеры группируют схожие по тематике обращения
- Silhouette score показывает качество кластеризации
- Статистика помогает понять распределение данных по кластерам
